In [1]:
import smtplib
import keyring
import sqlite3
import pandas as pd
from email.message import EmailMessage
from datetime import date
import csv
import os

In [2]:
# Games id that I'm interested in buying with a discount price
games_id = []

# Desired price
price_wanted = 17

with open('games.csv', 'r') as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        games_id.append(row[0])

pwd = os.getcwd()
con = sqlite3.connect(pwd+'/boardgames.db')
cur = con.cursor()
query = f'SELECT id, name, price, url, date FROM jugamosotra WHERE id IN (%s) AND date="{date.today()}"' % ','.join('?' for i in games_id)
cur.execute(query, games_id)
df = cur.fetchall()

In [3]:
df = pd.DataFrame(df, columns=['id', 'name', 'price','url' ,'date'])

In [4]:
df

,id,name,price,url,date


In [11]:
def send_mail():
    for i, game in df.iterrows():
        if game['price'] <= price_wanted:
            print(game['name'])
            print(game['price'])

            with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
                x = keyring.get_credential(service_name='mail_pess',username=None)
                user_mail = x.username
                pass_mail = x.password
                smtp.login(user_mail, pass_mail)

                msg = EmailMessage()
                msg['Subject'] = f"{game['name']} is at {game['price']}€! Now is your chance to buy!"
                msg['From'] = user_mail
                msg['To'] = user_mail
                msg.set_content(f"Filipe, This is the moment we have been waiting for.\nNow is your chance to pick up {game['name']}.\nDon't mess it up! Link here: {game['url']}")

                smtp.send_message(msg)
                print('Message sent! Check your e-mail account!')

In [12]:
send_mail()

Draftosaurus: Marina
11.65
Message sent! Check your e-mail account!
